# Исследование надёжности заёмщиков

Заказчиком этого проекты выступил кредитный отдел банка. 
Была поставлена задача разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования планируется участь при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

**В рамках исследования необходимо ответить на несколько вопросов:**

    Есть ли зависимость между наличием детей и возвратом кредита в срок?
    Есть ли зависимость между семейным положением и возвратом кредита в срок?
    Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
    Как разные цели кредита влияют на его возврат в срок?
    
В заключение сформулируем общий вывод по результатам исследования.

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [1]:
# сначала подключим библиотеки, которые понадобятся нам в процессе анализа данных
import pandas as pd # импортируем pandas - основной пакет, используемый при анализе
from pymystem3 import Mystem # импортируем лемматизатор для слов на русском
from collections import Counter # модуль Counter для подсчёта количества лем
m = Mystem()

In [2]:
# загрузим файл с данными 
df = pd.read_csv('this string intentionally replaces actual source')
df.info() # и изучим общую информацию

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Согласно документации к данным:

*    `children` — количество детей в семье
*    `days_employed` — общий трудовой стаж в днях
*    `dob_years` — возраст клиента в годах
*    `education` — уровень образования клиента
*    `education_id` — идентификатор уровня образования
*    `family_status` — семейное положение
*    `family_status_id` — идентификатор семейного положения
*    `gender` — пол клиента
*    `income_type` — тип занятости
*    `debt` — имел ли задолженность по возврату кредитов
*    `total_income` — ежемесячный доход
*    `purpose` — цель получения кредита

In [3]:
"""
смотрю на первые 10 строк датафрейма - хотелось бы сразу увидеть, 
что необычного в столбцах days_employed и total_income
"""
display(df.head(10))

# заодно взглянем и на последние 5
display(df.tail())

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля
21524,2,-1984.507589,40,среднее,1,женат / замужем,0,F,сотрудник,0,82047.418899,на покупку автомобиля


**Вывод**

В предоставленном для анализа файле:

    12 столбцов (переменных)
    21525 строк (записей/наблюдений)
    
Целочисленных и строковых переменных в таблице поровну: по 5 каждого вида. Ещё два столбца - **трудовой стаж в днях** и **общий доход** - вещественные числа.
Стиль в названии колонок не нарушен.

Вещественный тип данных в столбце `days_employed` вызывает подозрения - хранение дробных дней выглядит излишним.
При этом, в обоих столбцах не достаёт около двух тысяч наблюдений - вероятно, пропущенные данные хранятся как "вещественные числа" NaN.

Первые и последние строки таблицы не дали прямого ответа о наличии NaN в двух интересующих нас переменных, но ярко обозначили **наличие отрицательных значений** в общем трудовом стаже в днях. Также следует обратить внимание на слишком большие (по модулю) значения в этой переменной: уже в 4 строке наблюдаем запись с 340266 днями стажа - это 932 года. Значения нелогичны - нужно будет перепроверить. Гипотеза: стаж представлен в часах, а не днях.

Обращает на себя внимание и столбец "**education**" - значения в нём не приведены к одному регистру.

В каждой строке таблицы — данные об отдельном клиента банка. Большая часть колонок описывает самого клиента: возраст, пол, образование и т.д. Столбец `purpose` содержит информацию о цели кредита.

Предварительно можно утверждать, что, данных достаточно для проверки гипотез. Но в данных видны проблемы: отрицательные значения стажа, пропуски, разный регистр значений в строковых переменных.

Чтобы двигаться дальше, нужно устранить проблемы в данных.

## Шаг 2. Предобработка данных

### Обработка пропусков

Сначала подсчитаем количество пропусков в каждой колонке.

In [4]:
print(df.isna().sum())

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64


Равное количество пропусков в переменных `days_employed` и `total_income` наводит на мысль о том, что пропуски неслучайны. Взглянем на строки, в которых есть пропуски.

In [5]:
display(df[df['days_employed'].isna()])

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Среднее,1,женат / замужем,0,M,компаньон,0,NaN,сделка с автомобилем
21495,1,NaN,50,среднее,1,гражданский брак,1,F,сотрудник,0,NaN,свадьба
21497,0,NaN,48,ВЫСШЕЕ,0,женат / замужем,0,F,компаньон,0,NaN,строительство недвижимости
21502,1,NaN,42,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,строительство жилой недвижимости


Дополнительно проверим, во всех ли строках с пропуском стажа пропущена информация о доходе.

In [6]:
print(df[df['days_employed'].isna()]['total_income'].unique())

[nan]


Пропуски неслучайны - NaN в стаже соответствует NaN в ежемесячном доходе, во всех случаях без исключения.
Прежде, чем избавиться от пропусков, проверим, есть ли в интересующих нас колонках нулевые значения.

In [7]:
print(f"В столбце `days_employed` {df[df['days_employed'] == 0]['days_employed'].count()} нулевых значений")
print(f"В столбце `total_income` {df[df['total_income'] == 0]['total_income'].count()} нулевых значений")

В столбце `days_employed` 0 нулевых значений
В столбце `total_income` 0 нулевых значений


В обоих столбцах нулевые значения отсутствуют. Можем заменить пропуски в них на 0.

In [8]:
# заменяем значения NaN на 0
df = df.fillna(0)
# проверяем, остались ли в таблице пропуски
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Пропущенных значений в датафрейме не осталось. Подведём промежуточный итог.

**Вывод**

В предоставленных данных обнаружены пропущенные значения в столбцах `days_employed` (трудовой стаж в днях) и `total_income` (ежемесячный доход).
Пропуски неслучайны и могут быть заменены на 0 в обоих столбцах - у людей без трудового стажа нет дохода. В реальной ситуации было бы логично обратиться к источнику данных и уточнить, по возможности, причину пропусков. В учебном проекте, в контексте оценки кредитоспособности клиента отсутствие данных о стаже и доходе может быть приравнено к отсутствию стажа и дохода. 

Продолжим предварительную обработку данных и наведём порядок в типах данных.

### Замена типа данных

После избавления от значений NaN в столбцах `days_employed` (трудовой стаж в днях) и `total_income` (ежемесячный доход) можно изменить тип данных в этих столбцах - больше нет необходимости хранить "вещественный" NaN, а избыточная точность в числе знаков после запятой только затрудняет анализ.

Чтобы выбрать подходящий тип данных для каждого из столбцов, проверим, как распределены в них значения.

In [9]:
print(f"Минимальное значение трудового стажа: {df['days_employed'].min()} дней")
print(f"Максимальное значение трудового стажа: {df['days_employed'].max()} дней")

print(f"Минимальное значение ежемесячного дохода: {df['total_income'].min()} рублей")
print(f"Максимальное значение ежемесячного дохода: {df['total_income'].max()} рублей")

Минимальное значение трудового стажа: -18388.949900568383 дней
Максимальное значение трудового стажа: 401755.40047533 дней
Минимальное значение ежемесячного дохода: 0.0 рублей
Максимальное значение ежемесячного дохода: 2265604.028722744 рублей


Проверка максимальных и минимальных значений напомнила о том, что необходимо навести порядок с трудовым стажем - отрицательным он быть не может, равно как не может превышать разумную продолжительность жизни. Прежде чем заменить отрицательные значения в столбце проверим, сколько их и какому ежемесячному доходу они соответствуют.

In [10]:
print(f"В таблице {df[df['days_employed'] < 0]['days_employed'].count()} отрицательных значений трудового стажа.")
display(df[df['days_employed'] < 0][['days_employed', 'total_income']])

В таблице 15906 отрицательных значений трудового стажа.


,days_employed,total_income
0,-8437.673028,253875.639453
1,-4024.803754,112080.014102
2,-5623.422610,145885.952297
3,-4124.747207,267628.550329
5,-926.185831,255763.565419
...,...,...
21519,-2351.431934,115949.039788
21520,-4529.316663,224791.862382
21522,-2113.346888,89672.561153
21523,-3112.481705,244093.050500


Отрицательных значений стажа в днях много - при этом, им соответствуют ненулевые значения дохода.
Установить природу ошибки затруднительно. Учитывая количество строк с отрицательными значениями стажа - разумно предположить ошибку при выгрузке данных. Проверим дополнительно минимальное положительное и максимальное отрицательное значение стажа.

In [11]:
print(df[df['days_employed']>0]['days_employed'].min())
print(df[df['days_employed']<0]['days_employed'].max())

328728.72060451825
-24.14163324048118


Проверка показала, что в переменной `days_employed` нет значений по модулю меньших 24 - что равняется одному дню. Гипотеза об ошибочном представлении данных в часах подтверждается.

Можем внести необходимые коррективы: избавиться от отрицательных значений, привести значения к заявленному виду (дням) и изменить тип данных в переменных на целые числа.

In [12]:
# создадим функцию, которая приведёт значение в колонке к положительному количеству дней
def normalize_data(value):
    if value < 0:
        return value / (-24)
    return value / 24

df['days_employed'] = df['days_employed'].apply(normalize_data) # применим функцию ко всем значениям колонки `days_employed`

# преобразуем значения в столбцах `days_employed` и `total_income` в целые числа методом astype, дающим возможность указать
# конкретный тип переменной на выходе
df[['days_employed', 'total_income']] = df[['days_employed', 'total_income']].astype('int')

# проверим, сработало ли наше решение - снова обратимся к информации о df и посмотрим на первые строки таблицы
df.info()
display(df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,351,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,167,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,234,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,171,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


**Вывод**

Обнаружили две переменные, в которых данные были представленны в виде вещественных чисел: `days_employed` и `total_income`.
Преобразовали значения в целые числа методом `astype()`, дающим возможность указать необходимый конкретный тип значения на выходе.
Дополнительно обработали значения в переменной `days_employed` - избавились от отрицательных значений и привели к заявленному содержанию - дням вместо часов.

Можем приступить к обработке дубликатов.

### Обработка дубликатов

В предоставленной базе нет уникальных идентификаторов клиентов. Проверить дублирующиеся строки в этих условиях непросто. 
В любом случе, нашего внимания требует переменная `education` - в ней очевидно наличие разных регистров для записи одной и той же информации.

Также проверим остальные переменные типа object, хранящие строковые данные - `family_status` и `income_type`, на всякий случай взглянем на значения в столбце `gender`. Столбец `purpose` потребует дополнительной обработки.

Начнём с переменной `education` - приведём значения к одному регистру и проверим, какие значения `education` каким значения `education_id` соответствуют.

In [13]:
df['education'] = df['education'].str.lower()

print(df.groupby('education_id')['education'].unique())

education_id
0                 [высшее]
1                [среднее]
2    [неоконченное высшее]
3              [начальное]
4         [ученая степень]
Name: education, dtype: object


После обработки каждому значению `education_id` соответствует ровно одно значение `education`. Можно перейти к остальным столбцам.

In [14]:
for column in ['family_status', 'income_type', 'gender']:
    print(f"Уникальные значения в столбце {column}: {df[column].unique()}\n")

Уникальные значения в столбце family_status: ['женат / замужем' 'гражданский брак' 'вдовец / вдова' 'в разводе'
 'Не женат / не замужем']

Уникальные значения в столбце income_type: ['сотрудник' 'пенсионер' 'компаньон' 'госслужащий' 'безработный'
 'предприниматель' 'студент' 'в декрете']

Уникальные значения в столбце gender: ['F' 'M' 'XNA']



В столбцах `family_status` и `income_type` скрытых дубликатов не обнаружено. В столбце `gender` присутствует значение **XNA** - проверим, сколько таких строк и чем они отличаются от остальных.

In [15]:
display(df[df['gender'] == 'XNA'])

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
10701,0,98,24,неоконченное высшее,2,гражданский брак,1,XNA,компаньон,0,203905,покупка недвижимости


Строка, в которой переменная `gender` принимает значение **XNA** в наборе данных одна - её можно исключить из анализа.

In [16]:
df = df[df['gender'] != 'XNA']
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21524 entries, 0 to 21524
Data columns (total 12 columns):
children            21524 non-null int64
days_employed       21524 non-null int64
dob_years           21524 non-null int64
education           21524 non-null object
education_id        21524 non-null int64
family_status       21524 non-null object
family_status_id    21524 non-null int64
gender              21524 non-null object
income_type         21524 non-null object
debt                21524 non-null int64
total_income        21524 non-null int64
purpose             21524 non-null object
dtypes: int64(7), object(5)
memory usage: 2.1+ MB


Наконец, попробуем оценить существование проблем в переменной `purpose`. Подсчитаем количество уникальных значений и оценим, сколько ответов приходится на каждое из них.

In [17]:
print(f"Всего уникальных значений в purpose: {len(df['purpose'].unique())}")
print('')
print(df['purpose'].value_counts())

Всего уникальных значений в purpose: 38

свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
операции с жильем                         653
покупка жилья для сдачи                   653
операции с коммерческой недвижимостью     651
жилье                                     647
покупка жилья                             647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      623
покупка своего жилья                      620
строительство недвижимости                620
ремонт жилью                              612
покупка жилой недвижимости                607
на покупку своего автомобиля           

Очевидна необходимость дополнительной обработки значений в столбце `purpose` - для более подробного анализа необходимо провести лемматизацию.

**Вывод**

В процессе обработки дубликатов обнаружили и исправили расхождение регистров записи в переменной `education`, нашли уникальное значение **XNA** (вероятный пропуск) в переменной `gender` - строку с этим значением отфильтровали.
Подтвердили необходимость проведения лемматизации значений в столбце `purpose` для продолжения анализа и ответов на поставленные вопросы.

### Лемматизация

Проведение лемматизации необходимо для того, чтобы сгруппировать значения в переменной `purpose`. На первый взгляд видны несколько общих позиций:
* Получение образования
* Покупка автомобиля
* Сделки с недвижимостью (коммерческой и жилой - эти две позиции стоит разделять)
* Свадьба

Ручная обработка для перегруппировки 38 уникальных значений возможна, но при существовании автоматических инструментов неоправданна. 
Сохраним лемматизированные строки из переменной `purpose` для каждой строки в новую переменную `purpose_list`.

In [18]:
df_processed = df
df_processed['purpose_list'] = df['purpose'].apply(m.lemmatize)
print(df_processed['purpose_list'])

0                             [покупка,  , жилье, \n]
1                   [приобретение,  , автомобиль, \n]
2                             [покупка,  , жилье, \n]
3                [дополнительный,  , образование, \n]
4                           [сыграть,  , свадьба, \n]
                             ...                     
21520                  [операция,  , с,  , жилье, \n]
21521               [сделка,  , с,  , автомобиль, \n]
21522                              [недвижимость, \n]
21523    [на,  , покупка,  , свой,  , автомобиль, \n]
21524             [на,  , покупка,  , автомобиль, \n]
Name: purpose_list, Length: 21524, dtype: object


Необходимо также оценить, сколько значений каждой леммы получилось в наборе данных.
Для этого стоит составить список всех лемм, встречающихся в получившихся списках. 

In [19]:
# небольшой чит - методу explode() нас ещё не учили, но он удобен - разворачивает список.
print(Counter(df_processed['purpose_list'].explode()))

Counter({' ': 33676, '\n': 21524, 'недвижимость': 6366, 'покупка': 5911, 'жилье': 4473, 'автомобиль': 4315, 'образование': 4022, 'с': 2924, 'операция': 2610, 'свадьба': 2348, 'свой': 2235, 'на': 2233, 'строительство': 1881, 'высокий': 1375, 'получение': 1316, 'коммерческий': 1315, 'для': 1294, 'жилой': 1233, 'сделка': 944, 'дополнительный': 909, 'заниматься': 908, 'проведение': 777, 'сыграть': 774, 'сдача': 653, 'семья': 641, 'собственный': 635, 'со': 630, 'ремонт': 612, 'подержанный': 489, 'подержать': 479, 'приобретение': 462, 'профильный': 436})


Полученный список значений позволяет провести категоризацию данных.

**Вывод**

Провели лемматизацию значений переменной `purpose`, на выходе для каждой строки получили список с леммами, хранящийся в переменной `purpose_list`. После категоризации данных исходны и новый столбцы можно будет удалить.

### Категоризация данных

Сохранённые в `purpose_list` списки дают нам возможность категоризировать цели кредита. Для этого напишем функцию, которая на вход будет получать список `purpose_list` для каждой строки и на его основе присваивать ту или иную категорию.

Определимся с набором категорий:
* Образование
* Автомобиль
* Свадьба
* Ремонт (жилья)
* Жилая недвижимость
* Коммерческая и другая недвижимость

Номера категорий определим на отрезке [1:6].

Предусмотрим также значение для непредусмотренных случаев - **Other**, такому значению будет соответствовать NaN для облегчения оценки качества категоризации.

In [20]:
def categorize(row):
    try:
        if 'образование' in row['purpose_list']:
            return 'образование'
        if 'автомобиль' in row['purpose_list']:
            return 'автомобиль'
        if 'свадьба' in row['purpose_list']:
            return 'свадьба'
        if 'ремонт' in row['purpose_list']:
            return 'ремонт'
        if 'жилье' in row['purpose_list'] or 'недвижимость' in row['purpose_list']:
            if 'жилье' in row['purpose_list'] or 'жилой' in row['purpose_list']:
                return 'жилая недвижимость'
            return 'коммерческая и другая недвижимость'

    except:
        return float('nan')
    
df_processed['purpose_category'] = df_processed.apply(categorize, axis = 1)
print(df_processed['purpose_category'].value_counts())

коммерческая и другая недвижимость    5133
жилая недвижимость                    5094
автомобиль                            4315
образование                           4022
свадьба                               2348
ремонт                                 612
Name: purpose_category, dtype: int64


Присвоим получившимся категориям заранее определённые коды (в порядке их определения).

In [21]:
def encode(row):
    if 'образование' in row['purpose_category']:
        return 1
    if 'автомобиль' in row['purpose_category']:
        return 2
    if 'свадьба' in row['purpose_category']:
        return 3
    if 'ремонт' in row['purpose_category']:
        return 4
    if 'жилая недвижимость' in row['purpose_category']:
        return 5
    if 'коммерческая и другая недвижимость' in row['purpose_category']:
        return 6
    return float('nan')

df_processed['purpose_id'] = df_processed.apply(encode, axis = 1)
df_processed.drop(labels = ['purpose', 'purpose_list'], axis = 1)
print(df_processed['purpose_id'].value_counts())

6    5133
5    5094
2    4315
1    4022
3    2348
4     612
Name: purpose_id, dtype: int64


Исходя из вопросов, поставленных в исследовании, категоризируем также переменную `children` - в исходном виде в переменной сохранено количество детей, а нас интересует сам факт их наличия.

In [22]:
def children_presence(row):
    try:
        if row['children'] > 0:
            return 'exist'
        return 'not exist'
    except:
        return float('nan')
df_processed['children_exist'] = df_processed.apply(children_presence, axis = 1)
print(df_processed['children_exist'].value_counts())

not exist    14195
exist         7329
Name: children_exist, dtype: int64


Также нам необходимо категоризировать значения уровня дохода, чтобы ответить на вопрос о влиянии дохода на возврат кредита в срок. Для начала оценим разброс значений в столбце - проверим минимальное отличное от нуля значение, медиану, среднее и максимум.

In [23]:
print('Минимальное значение ненулевого дохода: ', df_processed[df_processed['total_income'] > 0]['total_income'].min())
print('Медианное значение ненулевого дохода: ', df_processed[df_processed['total_income'] > 0]['total_income'].sort_values().median())
print('Среднее значение ненулевого дохода: ', df_processed[df_processed['total_income'] > 0]['total_income'].mean())
print('Максимальное значение ненулевого дохода: ', df_processed[df_processed['total_income'] > 0]['total_income'].max())

Минимальное значение ненулевого дохода:  20667
Медианное значение ненулевого дохода:  145011.0
Среднее значение ненулевого дохода:  167419.91627906976
Максимальное значение ненулевого дохода:  2265604


Среднее значение ненулевого дохода несколько завышено - но надо сказать, что мы избавились от нулевых значений при его расчёте, а также отметить, что для среднего значения дохода это довольно стандартная ситуация.

Воспользуемся этими четырьмя значениями, чтобы создать категориальную переменную. Определим следующие категории:
* `total_income` = 0 - нет дохода или данные отсутствуют
* 0 < `total_income` < медианного значения - низкий доход
* Медианное значение <= `total_income` <= среднее значение - средний доход
* `total income` > среднего значения - высокий доход

In [24]:
income_median = df_processed[df_processed['total_income'] > 0]['total_income'].sort_values().median()
income_mean = df_processed[df_processed['total_income'] > 0]['total_income'].mean()

def income_cat(row):
    if row['total_income'] == 0:
        return 'Нулевой / нет данных'
    elif row['total_income'] < income_median:
        return 'Низкий доход'
    elif row['total_income'] <= income_mean:
        return 'Средний доход'
    return 'Высокий доход'

df_processed['income_category'] = df_processed.apply(income_cat, axis = 1)
display(df_processed[['total_income', 'income_category']])

,total_income,income_category
0,253875,Высокий доход
1,112080,Низкий доход
2,145885,Средний доход
3,267628,Высокий доход
4,158616,Средний доход
...,...,...
21520,224791,Высокий доход
21521,155999,Средний доход
21522,89672,Низкий доход
21523,244093,Высокий доход


**Вывод**

Лемматизировали переменную `purpose`, выделили 6 категорий кредитов, сохранив их в `purpose_category`. Каждой категории присвоили номер и сохранили его в `purpose_id`.

Категоризировали наличие детей и показатель дохода.

Можно приступать к следующему шагу: ответить на вопросы исследования.

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [25]:
children_pivot = df_processed.pivot_table(index = 'children_exist', values = 'debt', aggfunc = ['count', 'sum'])
children_pivot['ratio'] = children_pivot['sum'] / children_pivot['count']
display(children_pivot)

,count,sum,ratio
,debt,debt,
children_exist,,,
exist,7329,677,0.092373
not exist,14195,1064,0.074956


**Вывод**

Среди клиентов с детьми 9% имеют просроченный кредит, тогда как среди заёмщиков без детей доля просрочек составляет 7.5%. Зависимость между наличием детей и возвратом кредита в срок существует *(хорошо бы проверить результат z-тестом, например, но с учётом размера выборки 1.5 п.п. разницы на таком удалении от 50% вероятно будут значимы)*.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [26]:
family_status_pivot = df_processed.pivot_table(index = 'family_status', values = 'debt', aggfunc = 'sum') \
                        / df_processed.pivot_table(index = 'family_status', values = 'debt', aggfunc = 'count')
display(family_status_pivot)

,debt
family_status,
Не женат / не замужем,0.097405
в разводе,0.071130
вдовец / вдова,0.065625
гражданский брак,0.092912
женат / замужем,0.075202


**Вывод**

Наиболее высокий процент невозврата кредита в срок наблюдается у клиентов, не состоящих в браке или сожительствующих (*термин "гражданский брак" неудачный - он описывает собственно зарегистрированный в органах ЗАГС брак, в противоположность или дополнение церковному*). 

Среди разведённых и находящихся в браке доля должников по кредитам около 7%, а рекордсмены по возвратам - вдовцы и вдовы, здесь доля просроченных платежей составляет 6.5%.

Зависимость между семейным положением и возвратом кредита в срок есть (*учтём здесь, что на возврат кредита в срок может влиять возраст заёмщика - с высокой долей вероятности, именно молодёжь не состоит в браке или сожительствует, тогда как в остальных группах средний возраст весьма вероятно выше*).

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [27]:
income_pivot = df_processed.pivot_table(index = 'income_category', values = 'debt', aggfunc = 'sum') \
                        / df_processed.pivot_table(index = 'income_category', values = 'debt', aggfunc = 'count')
display(income_pivot)

,debt
income_category,
Высокий доход,0.076160
Низкий доход,0.083101
Нулевой / нет данных,0.078197
Средний доход,0.089761


**Вывод**

Согласно предоставленным данным, зависимость между доходом и возвратом кредита в срок есть, хотя и не очень значительная. Заёмщики с низким и средним доходом с несколько большей вероятностью имеют просроченный кредит.

Любопытно, что заёмщики, о доходе которых данных нет в целом показыают сравнительно низкую долю кредитов, не возвращённых в срок - 7.8%. Лучше них выглядят только люди с высоким доходом - среди них доля просроченных кредитов - 7.6%.

- Как разные цели кредита влияют на его возврат в срок?

In [28]:
purpose_pivot = df_processed.pivot_table(index = 'purpose_category', values = 'debt', aggfunc = 'sum') \
                        / df_processed.pivot_table(index = 'purpose_category', values = 'debt', aggfunc = 'count')
display(purpose_pivot)

,debt
purpose_category,
автомобиль,0.093395
жилая недвижимость,0.071064
коммерческая и другая недвижимость,0.075005
образование,0.091994
ремонт,0.057190
свадьба,0.079216


**Вывод**

Наилучший уровень возврата в срок у кредитов, выданных на ремонт - лишь 5.7% кредитов с этой целью просрочены. За ними следует жилая недвижимость (7.1%), а замыкает тройку лидеров коммерческая недвижимость (7.5%).

Доля просроченных кредитов на свадьбу составляет 7.9% - мы уже знаем, что люди, состоящие в браке, более склонны платить вовремя.

Антилидеры рейтинга - кредиты на автомобиль и образование. В обоих случаях доля просроченных кредитов превышает 9%. Здесь я радуюсь за себя - на обучение в Практикуме брать кредит не пришлось, а до получения прав ещё несколько месяцев. По крайней мере, в ближайшее время мне не грозит оказаться тем единственным должником из 11 заёмщиков. 

## Шаг 4. Общий вывод

В ходе исследования надёжности заёмщиков мы провели предварительную обработку предоставленных данных и проверили наличие зависимости между некоторыми характеристиками заёмщиков (либо самих кредитов) и возвратом кредита в срок.

По каждому показателю зависимость так или иначе обнаружена:
* Невозврат кредита в срок более распространён среди тех заёмщиков, кто ещё не создал полноценную семью и либо сожительствует с партнёром, либо не состоит в постоянных отношениях.
* При этом, дети в семье делают возврат кредита в срок более сложной задачей - доля просроченных кредитов среди заёмщиков с детьми выше, чем среди бездетных, на 1.5 процентных пункта.
* Заёмщики с высоким доходом более склонны возвращать кредиты в срок в сравнение с заёмщиками со средними или низкими доходами. Любопытно, что для тех случаев, когда у банка нет данных о доходах, также наблюдается меньшая доля просроченных кредитов.
* Самые безопасные для банка кредиты связаны с недвижимостью: кредиты взятые на ремонт, покупку жилья или коммерческой недвижимости (вероятно, в силу обеспечения этой самой недвижимостью) возвращаются лучше, чем кредиты на автомобиль или образование.